In [ ]:
%reload_ext autoreload
%autoreload 2

from pathlib import Path

from sklearn.model_selection import train_test_split

from tscls.dataloaders import TimeSeriesLoader

data_paths = {
    0: Path("../data/Si_wafer_clear_SRRs"),
    1: Path("../data/SRRs_cleaned_with_ethanol"),
    2: Path("../data/SRRs_with_10ppb_Acetamiprid"),
    3: Path("../data/SRRs_with_100ppb_Acetamiprid"),
    4: Path("../data/SRRs_with_1000ppb_Acetamiprid"),
    5: Path("../data/SRRs_with_10000ppb_Acetamiprid"),
    6: Path("../data/SRRs_with_100000ppb_Acetamiprid"),
}

tsl = TimeSeriesLoader(data_paths)

X, y = tsl.load()

tsl.report()

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [ ]:
from aeon.classification.convolution_based import RocketClassifier
from tsai.all import (
    InceptionTime,
    InceptionTimePlus,
    ResNet,
    XceptionTime,
)

from tscls.experiments import AeonExperiment, TsaiExperiment

tsai_exp = TsaiExperiment(X=X, y=y, random_state=42)
aeon_exp = AeonExperiment(X=X, y=y, random_state=42)

# OmniScaleCNN excluded because it is very slow to train
results = [
    tsai_exp.train(InceptionTimePlus, "InceptionTimePlus", epochs=100),
    tsai_exp.train(InceptionTime, "InceptionTime", epochs=100),
    tsai_exp.train(ResNet, "ResNet", epochs=100),
    tsai_exp.train(XceptionTime, "XceptionTime", epochs=100),
]

results += [aeon_exp.train(RocketClassifier, "Rocket", n_kernels=10000)]

tsai_exp.print_summary(results)